In [1]:
%load_ext autoreload
%autoreload 2

# Simple IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [43]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d')

,change,changeOverTime,changePercent,close,high,label,low,open,uClose,uHigh,uLow,uOpen,uVolume,volume
date,,,,,,,,,,,,,,
2019-09-30,0.00,0.000000,0.0000,231.48,234.64,Sep 30,221.81,230.30,229.26,228.03,230.10,225.40,26836800,26634345
2019-10-01,0.63,0.002813,0.2860,230.43,232.22,Oct 1,228.40,231.13,233.81,229.73,226.10,225.33,37549879,36964121
2019-10-02,-5.78,-0.022770,-2.5135,228.61,229.02,Oct 2,225.29,232.71,224.71,233.20,228.66,232.18,36373450,36082815
2019-10-03,1.88,-0.014334,0.8665,231.38,225.03,Oct 3,219.88,219.20,224.89,229.04,215.48,222.41,31733834,30957763
2019-10-04,6.21,0.014214,2.9104,236.71,230.05,Oct 4,231.56,228.99,234.99,230.43,227.37,229.44,35793765,36380802


In [44]:
aapl.chart('5d', chartCloseOnly=True)

,change,changeOverTime,changePercent,close,volume
date,,,,,
2019-09-30,0.00,0.000000,0.0000,232.70,27406280
2019-10-01,0.64,0.002789,0.2904,234.61,37247422
2019-10-02,-5.82,-0.023449,-2.6087,223.08,36111255
2019-10-03,1.90,-0.014263,0.8565,231.56,31266500
2019-10-04,6.28,0.013826,2.8464,228.54,36016338


## Fetcher SPY stocks

In [3]:
import pandas as pd
spy = [x for x in pd.read_html('https://etfdailynews.com/etf/spy/', attrs={'id': 'etfs-that-own'})[0].Symbol.values.tolist() if isinstance(x, str)]
correction = {'PCLN':'BKNG', 'DWDP':'DD'}

In [4]:
import os, sys
chart_range = [(5,'break'), (10,'1m'), (45,'3m'), (90,'6m'), (250,'1y'), (99999,'max')]

In [ ]:
# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

# save historical price to disk
for k in spy:
    if k in correction:
        k = correction[k]
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    if os.path.exists(filename):
        # fetcher missing period
        df = pd.read_csv(filename, index_col='date', parse_dates=True)
        td = pd.Timedelta(df.index[-1] - pd.datetime.now())
        for day, param in chart_range:
            if td.days < day:
                if param != 'break':
                    df_append = iex.Stock(k).chart(param)
                    pd.concat([df, df_append]).to_csv(filename)
                break
    else:
        iex.Stock(k).chart('max').to_csv(filename)
    
